# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.csv

pub_date,title,venue,excerpt,citation,url_slug,paper_url
1/2/20,"Global irrigation characteristics and effects simulated by fully coupled land surface, river, and water management models in E3SM",Journal of Advances in Modeling Earth Systems,"This paper introduces a two_way coupled irrigation scheme developed within the Energy Exascale Earth System Model (E3SM). The new irrigation scheme simulates irrigation water demand and applies irrigation water in E3SM Land Model (ELM), which is coupled to a river routing model and a water management model (MOSART_WM) that simulate streamflow, reservoir operations, and irrigation water supply",,E3SM-irrigation,https://doi.org/10.1029/2020MS002069
1/4/20,Impact of climate change on water availability and its propagation through the Western U.S. power grid,Applied Energy,This paper evaluates how projected changes in water availability affect electricity generation at hydroelectric and thermal power plants and how the coincident impacts propagate loc

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [11]:
publications = pd.read_csv("add_new_pub_here.csv", sep=",", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,9/1/2022,The Third Assessment of the Effects of Climate...,Report to U.S. Congress,This report evaluates the effects of climate c...,"Kao, Shih-Chieh, Ashfaq, Moetasim, Rastogi, De...",9505v3,https://doi.org/10.2172/1887712


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [12]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [17]:
import os
for row, item in publications.iterrows():
    
    year = item.pub_date[4:8]
    md_filename = str(year) + "-" + item.url_slug + ".md"
    html_filename = str(year) + "-" + item.url_slug
        
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Link to the paper](" + item.paper_url + ")\n" 
    
    md += "\n![image](../images/papers/" + str(year) + "-" + item.url_slug + ".png)\n"
    
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)
        
    item.pub_date

In [18]:
item.pub_date[4:8]

'2022'

These files are in the publications directory, one directory below where we're working from.

In [15]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
